In [3]:
import cv2
import numpy as np 
import torch
from wider import WIDER
from PIL import Image
import tqdm

In [27]:
def unmain(wider):

    annot_dict = {}

    # press ctrl-C to stop the process
    for idx, data in tqdm.tqdm(enumerate(wider.next())):
        # im = cv2.imread(data.image_name)
        im = Image.open(data.image_name)
        im_id = data.image_name.split('/')[-1]

        if im_id not in annot_dict.keys():
            # Create entry for current image
            annot_dict[im_id] = []

        width, height = im.size
        for bbox in data.bboxes:
            x1 = bbox[0]
            y1 = bbox[1]
            x2 = bbox[2]
            y2 = bbox[3]

            w = x2 - x1
            h = y2-y1
            xc = (x1 + (w/2))/width
            yc = (y1 + (h/2))/height
            w /= width
            h /= height

            annot_dict[im_id].append([0, xc, yc, w, h])
    return annot_dict

In [28]:
wider = WIDER('./wider_face_split',
          './WIDER_train/images',
          'wider_face_train.mat')

val_wider = WIDER('./wider_face_split',
          './WIDER_val/images',
          'wider_face_val.mat')

In [29]:
annot_dict = unmain(wider)
val_annot_dict = unmain(val_wider)

12880it [00:03, 4183.26it/s]
3226it [00:01, 2034.06it/s]


In [35]:
def write_to_files(annot_dict, location="wider_annots"):
    # Here do a final run over the dictionary and save to txt files
    # Annotations output file
    
    # Here we write the image_locations to the all.txt file
    list_file = open("all.txt", "a+")
    for image_id in tqdm.tqdm(annot_dict.keys()):
        list_file.write(f"data/{location}/{image_id}\n")

#         # Writing annotations to individual txt files
#         out_path = f"{location}/{image_id}.txt"
#         with open(out_path, 'w+') as out_file:
#             # For annotations for a single image
#             for ann in annot_dict[image_id]:
#                 # Write this annotation to the file
#                 out_file.write(" ".join([str(a) for a in ann]) + '\n')

    print("Done writing annotations to file")
    list_file.close()

In [36]:
write_to_files(annot_dict)
write_to_files(val_annot_dict)

100%|██████████| 3226/3226 [00:00<00:00, 383874.96it/s]

Done writing annotations to file
Done writing annotations to file
